In [96]:
!kaggle datasets list -s airplane

ref                                                           title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
saurograndi/airplane-crashes-since-1908                       Airplane Crashes Since 1908                         564KB  2016-09-09 20:43:04          33383        430  1.0              
cgurkan/airplane-crash-data-since-1908                        Airplane Crash Data Since 1908                      621KB  2019-08-20 12:24:25           5671        125  1.0              
thedevastator/airplane-crashes-and-fatalities                 Airplane Crashes and Fatalities                     582KB  2022-09-20 05:30:35           3935         96  1.0              
zernach/2018-airplane-flights                                 2018 Air

In [97]:
!kaggle datasets download thedevastator/airplane-crashes-and-fatalities

airplane-crashes-and-fatalities.zip: Skipping, found more recently modified local copy (use --force to force download)


In [125]:
!streamlit run case_2.py

^C


In [98]:
import pandas as pd
import zipfile 
!dir

zf = zipfile.ZipFile("./airplane-crashes-and-fatalities.zip")
acf_df = pd.read_csv(zf.open("Airplane_Crashes_and_Fatalities_Since_1908.csv"))
acf_df.head()

 Volume in drive C is Windows
 Volume Serial Number is DA1F-1EE1

 Directory of c:\HBO Aviation\Jaar 3\Minor\Github\DS-groep-6\Case-2

08-03-2023  12:06    <DIR>          .
08-03-2023  12:06    <DIR>          ..
08-03-2023  12:06           219.566 airlines-traffic-passenger-statistics.zip
08-03-2023  12:06           596.065 airplane-crashes-and-fatalities.zip
08-03-2023  12:55            87.163 case_2.ipynb
08-03-2023  12:35             4.290 case_2.py
               4 File(s)        907.084 bytes
               2 Dir(s)  106.592.043.008 bytes free


,index,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [99]:
import seaborn as sns 
import matplotlib as plt
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots
from datetime import date

In [100]:
# total fatilities of each year 
acf_df['Date'] = pd.to_datetime(acf_df['Date'])

# jaarlijkse slachtoffers 
yearly_fatalities = acf_df.groupby(acf_df.Date.dt.year).agg({'Fatalities': 'sum', 'Aboard': 'sum'}).reset_index()


In [101]:
aircraft_counts = acf_df.groupby([acf_df.Date.dt.year, 'Type']).size().reset_index(name = "aantal") # Aantal vliegtuigen per type
max_ac_per_year = aircraft_counts.groupby('Date')['aantal'].idxmax() # Meest voorkomend vliegtuig per jaar op basis van index
most_common_aircraft = aircraft_counts.loc[max_ac_per_year] # Laat het type vliegtuig zien dat het vaakst voorkomt per jaar. 

yearly_fatalities_aircraft = yearly_fatalities.merge(most_common_aircraft[["Date", "Type"]], on = "Date")


In [102]:
# creeëren van slider
rangeselector=dict(
    buttons=list([
        dict(count=10, label = "10j", step = "year", stepmode = "backward"),
        dict(count=20, label = "20j", step = "year", stepmode = "backward"),
        dict(count=30, label = "30j", step = "year", stepmode = "backward"),
        dict(count=40, label = "40j", step = "year", stepmode = "backward"),
        dict(count=50, label = "50j", step = "year", stepmode = "backward"),
        dict(count=60, label = "60j", step = "year", stepmode = "backward"),
        dict(count=70, label = "70j", step = "year", stepmode = "backward"),
        dict(count=80, label = "80j", step = "year", stepmode = "backward"),
        dict(count=90, label = "90j", step = "year", stepmode = "backward"),
        dict(count=100, label = "100j", step = "year", stepmode = "backward"),
        dict(step = "all")]))



In [103]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = yearly_fatalities_aircraft["Date"], y = yearly_fatalities_aircraft["Fatalities"], name = "Slachtoffers"))
fig.add_trace(go.Scatter(x = yearly_fatalities_aircraft["Date"], y = yearly_fatalities_aircraft["Aboard"], name = "Aanboord"))

fig.update_layout(title = "Totaal aantal passagiers en slachtoffers", xaxis_title = "Jaren", yaxis_title = "Aantal passagiers en slachtoffers",
                 xaxis_rangeslider_visible=True, xaxis=dict(rangeselector=rangeselector),
                  updatemenus = [{"type":'dropdown',"showactive" : True, "active":0
                               }]
                 )
                  
                 

fig.show()

In [104]:
yearly_fatalities_aircraft.head()

,Date,Fatalities,Aboard,Type
0,1908,1.0,2.0,Wright Flyer III
1,1912,5.0,5.0,Dirigible
2,1913,45.0,51.0,Curtiss seaplane
3,1915,40.0,60.0,Zeppelin L-10 (airship)
4,1916,108.0,109.0,Schutte-Lanz S-L-10 (airship)


In [105]:
yearly_fatalities.sort_values("Fatalities", ascending = False).iloc[0:10]

,Date,Fatalities,Aboard
60,1972,2937.0,3635.0
73,1985,2670.0,3480.0
61,1973,2485.0,3433.0
84,1996,2386.0,3058.0
62,1974,2318.0,3064.0
77,1989,2293.0,3643.0
56,1968,2156.0,2928.0
65,1977,2138.0,2662.0
80,1992,2122.0,2965.0
64,1976,2059.0,2629.0


In [106]:
yearly_fatalities_aircraft["Type"].value_counts().iloc[0:10]

Douglas DC-3                                32
de Havilland Canada DHC-6 Twin Otter 300     8
De Havilland DH-4                            6
Breguet 14                                   6
Antonov AN-26                                5
Cessna 208B Grand Caravan                    3
Boeing 40                                    2
Douglas C-47                                 2
Junkers JU-52/3m                             2
Cessna 208B Caravan I Super Cargomaster      2
Name: Type, dtype: int64

In [107]:
yearly_fatalities.describe()

,Date,Fatalities,Aboard
count,98.000000,98.000000,98.000000
mean,1960.438776,1076.316327,1475.010204
std,28.541547,779.428665,1094.199773
min,1908.000000,1.000000,2.000000
25%,1936.250000,194.750000,227.250000
50%,1960.500000,1214.000000,1504.500000
75%,1984.750000,1629.750000,2392.750000
max,2009.000000,2937.000000,3643.000000


In [108]:
top_10_ac = acf_df["Type"].value_counts().iloc[0:10]
print(top_10_ac)

Douglas DC-3                                334
de Havilland Canada DHC-6 Twin Otter 300     81
Douglas C-47A                                74
Douglas C-47                                 62
Douglas DC-4                                 40
Yakovlev YAK-40                              37
Antonov AN-26                                36
Junkers JU-52/3m                             32
Douglas C-47B                                29
De Havilland DH-4                            28
Name: Type, dtype: int64


In [109]:
# inladen andere dataset
!kaggle datasets download thedevastator/airlines-traffic-passenger-statistics

airlines-traffic-passenger-statistics.zip: Skipping, found more recently modified local copy (use --force to force download)


In [110]:
zf1 = zipfile.ZipFile("./airlines-traffic-passenger-statistics.zip")
atps_df = pd.read_csv(zf1.open("Air_Traffic_Passenger_Statistics.csv"))
atps_df.head()

,index,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count,Adjusted Activity Type Code,Adjusted Passenger Count,Year,Month
0,0,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,B,27271,Deplaned,27271,2005,July
1,1,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,B,29131,Enplaned,29131,2005,July
2,2,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,Terminal 1,B,5415,Thru / Transit * 2,10830,2005,July
3,3,200507,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Other,Terminal 1,B,35156,Deplaned,35156,2005,July
4,4,200507,Air Canada,AC,Air Canada,AC,International,Canada,Enplaned,Other,Terminal 1,B,34090,Enplaned,34090,2005,July


In [111]:
atps_df["Date"] = atps_df["Year"].copy()
print(atps_df["Date"])

0        2005
1        2005
2        2005
3        2005
4        2005
         ... 
15002    2016
15003    2016
15004    2016
15005    2016
15006    2016
Name: Date, Length: 15007, dtype: int64


In [112]:
yearly_passenger_count = atps_df.groupby('Date')['Passenger Count'].sum().reset_index()
print(yearly_passenger_count)

    Date  Passenger Count
0   2005         17222033
1   2006         33332970
2   2007         35554082
3   2008         37234678
4   2009         37338942
5   2010         39253999
6   2011         40927786
7   2012         44399885
8   2013         44945760
9   2014         47114631
10  2015         50057887
11  2016         11429847


In [113]:
fatalities_05_09 = yearly_fatalities[yearly_fatalities["Date"] >= 2005]
print(fatalities_05_09)

    Date  Fatalities  Aboard
93  2005      1306.0  2164.0
94  2006      1136.0  1413.0
95  2007       931.0  1364.0
96  2008       820.0  1463.0
97  2009       581.0   891.0


In [114]:
passengercount_05_09 = yearly_passenger_count[ (yearly_passenger_count["Date"] >= 2005) & (yearly_passenger_count["Date"] <= 2009)]
print(passengercount_05_09)

   Date  Passenger Count
0  2005         17222033
1  2006         33332970
2  2007         35554082
3  2008         37234678
4  2009         37338942


In [115]:
passenger_fatalities = passengercount_05_09.merge(fatalities_05_09, on = "Date")
print(passenger_fatalities)

   Date  Passenger Count  Fatalities  Aboard
0  2005         17222033      1306.0  2164.0
1  2006         33332970      1136.0  1413.0
2  2007         35554082       931.0  1364.0
3  2008         37234678       820.0  1463.0
4  2009         37338942       581.0   891.0


In [116]:
fig2 = make_subplots(rows = 2, cols = 1, subplot_titles=("Aantal passagiers", "Aantal slachtoffers"))


fig2.append_trace(go.Bar(x = passenger_fatalities["Date"], y = passenger_fatalities["Passenger Count"], name = "Aantal passagiers"), row = 1, col = 1)
fig2.append_trace(go.Bar(x = passenger_fatalities["Date"], y = passenger_fatalities["Fatalities"], name = "Aantal slachtoffers", offsetgroup=1), row = 2, col = 1)



fig2.update_layout(title_text="Aantal passagiers en slachtoffers 2005-2009")
fig2.update_xaxes(title_text="Jaren")
   

fig2.show()